In [1]:
from itertools import accumulate
# import lpips
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as dset
import torchvision
from torch import no_grad
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
%run unet.ipynb

torch.Size([16, 1, 32, 32])


In [2]:
def update_theta_mius(f_theta, f_theta_mius, mu):
    with torch.no_grad():
        for para in zip(f_theta_mius.parameters(), f_theta.parameters()):
           para[0].data = mu*para[0].data + (1-mu)*para[1].data
    # return mu*theta_mius + (1-mu)*theta

In [3]:
batch = 128
epoch = 500
DOWNLOAD_MNIST = False
device_str = "cuda" if torch.cuda.is_available() else "cpu"
# device_str = "cpu"
resize = 32
save_name = "cm_2"
device = torch.device(device_str)
print(device)

cuda


In [4]:
# cm para
s1 = 1000
s0 = 10
mu0 = 0.99
K = epoch
# N = lambda k: np.ceil(np.sqrt((k / K * ((s1 + 1) ** 2 - s0 ** 2) + s0 ** 2)) - 1) + 1
N = lambda k: 80
# mu = lambda k: np.exp(s0 * np.log(mu0) / N(k))
mu = lambda k: 0.99

epsilon = 0.002
T = 80
ro = 7
# time_schedule = lambda i: (epsilon**(1/ro) + (i-1)/(N-1)*(T**(1/ro) - epsilon**(1/ro)))**ro
time_schedule = lambda i, k: (epsilon ** (1 / ro) + (i - 1) / (N(k) - 1) *(
            T ** (1 / ro) - epsilon ** (1 / ro))) ** ro

In [5]:
data_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    # transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]
)

trainset = torchvision.datasets.MNIST(
    root = 'dataset/mnist',
    train = True,
    transform = data_transform, #改成torch可讀
    download = DOWNLOAD_MNIST,
)


In [6]:
trainLoader = dset.DataLoader(trainset, batch_size=batch, shuffle=True)
lr = 1e-2
f_theta = Unet().to(device)
f_theta_mius = Unet().to(device)
f_theta_mius.load_state_dict(f_theta.state_dict())
optimizer = torch.optim.Adam(f_theta.parameters(), lr=lr)
loss_func = nn.MSELoss()
loss_set = []
batch_loss = []

### Load model
# f_theta = Unet().to(device)
# checkpoint = torch.load(f"weights/eps_theta_{save_name}.pt", weights_only=False)
# f_theta.load_state_dict(checkpoint['model_state_dict'])
# f_theta_mius = Unet().to(device)
# f_theta_mius.load_state_dict(f_theta.state_dict())
# loss_set = checkpoint['loss_set']
# optimizer = torch.optim.Adam(f_theta.parameters(), lr=lr)
# loss_func = nn.MSELoss()

# loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
# loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

# image should be RGB, IMPORTANT: normalized to [-1,1]
# d = loss_fn_alex(img0, img1)

pbar = tqdm(range(epoch))
# mu = 0.1
# accumulation_steps = 4
for ep in pbar:
    # for ep in range(epoch):
    for i, (X_data, _) in enumerate(trainLoader):
        # with torch.autocast(device_type=device_str, dtype=torch.bfloat16):

        X_data = X_data.to(device)
        # X_data = 2 * X_data.to(device) - 1
        # index = torch.randint(N, size=(X_data.shape[0],))
        index = torch.randint(int(N(ep)), size=(X_data.shape[0],))

        # forward of DDPM
        noise = torch.randn_like(X_data)
        time_next = time_schedule(index + 1, ep)[:, None, None, None].to(device)
        time = time_schedule(index, ep)[:, None, None, None].to(device)
        # Xt = X_data + (2 * time).sqrt() * noise

        Xt = X_data + time * noise
        Xt_next = X_data + time_next * noise

        c_skip = lambda t: 0.25 / ((t - epsilon) ** 2 + 0.25)
        c_out = lambda t: 0.5 * (t - epsilon) / (t ** 2 + 0.25).sqrt()
        f1 = c_skip(time_next) * Xt_next + c_out(time_next) * f_theta(Xt_next, time_schedule(index + 1, ep).to(
            device))  # 餵給net吃訓練數據x, 輸出預測值
        with no_grad():
            f2 = c_skip(time) * Xt + c_out(time) * f_theta_mius(Xt, time_schedule(index, ep).to(
                device))  # 餵給net吃訓練數據x, 輸出預測值


        loss = 1 * loss_func(f1, f2)

        # loss = loss /accumulation_steps
        loss.backward()  # 誤差反向傳導
        optimizer.step()  # 神經網路參數更新
        optimizer.zero_grad()  # 梯度清0
        mu_k = mu(ep)
        update_theta_mius(f_theta, f_theta_mius, mu_k)


        # if (i+1) % accumulation_steps == 0:
            # optimizer.step()  # 神經網路參數更新
            # optimizer.zero_grad()  # 梯度清0
            # mu_k = mu(ep)
            # update_theta_mius(f_theta, f_theta_mius, mu_k)
        pbar.set_description('Loss: {}'.format(loss.item()))  # 更新pbar
        # print('Loss{}: {}'.format(loss.item()))
        # lr = 8e-5 if loss<0.02 else 1e-3
        loss_set.append(loss.item())

iteration = np.arange(len(loss_set))
fig = plt.figure()
plt.loglog(iteration, loss_set, "-")
# plt.show()

# try:
#     checkpoint
# except NameError:
#     epoch_last = 0
# else:
#     epoch_last = checkpoint["epoch"]
torch.save({
    # 'statement':
    # f"This model is U-net structure and DDPM process with linear schedule. \n
    #   It trained for {epoch_last + epoch} epoches with {N} steps. \n
    #   The dictionary save the model, trained parameter of model, Adam optimizer and loss. \n
    #   You can keep training for your goal with the dictionary."
    'epoch': epoch,
    # 'step': N,
    'model': f_theta,
    'model_state_dict': f_theta.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss_set': loss_set
}, f"weights/eps_theta_{save_name}.pt")
iteration = np.arange(len(loss_set))
fig = plt.figure()
plt.loglog(iteration, loss_set, "-")
plt.savefig(f"{save_name}.png")
plt.close()

  0%|          | 0/500 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 23.63 GiB of which 77.19 MiB is free. Process 3074860 has 22.50 GiB memory in use. Process 3097273 has 980.00 MiB memory in use. Of the allocated memory 451.90 MiB is allocated by PyTorch, and 62.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
torch.set_float32_matmul_precision('high')
Total_data = 50000
# batch = 64#5000
dtype = torch.float32
Total_iter = Total_data // batch + (Total_data % batch != 0)
device_str = "cuda" if torch.cuda.is_available() else "cpu"
# model_weight_name = "cm_cifar10_6"
device = torch.device(device_str)
print(device)

In [ ]:
f_theta = Unet().to(device)
checkpoint = torch.load(f"weights/eps_theta_{save_name}.pt", weights_only=False)
f_theta.load_state_dict(checkpoint['model_state_dict'])
f_theta = f_theta.eval()
f_theta = torch.compile(f_theta.to(device, dtype=dtype),fullgraph=True)
loss_set = checkpoint['loss_set']
print(len(loss_set))

In [ ]:
# pbar = tqdm(range(Total_iter))
pbar = tqdm(range(1))
for k in pbar:
    ep = 1000
    stard_id = k * batch
    # index = torch.randint(int(N(ep)), size=(int(batch),))
    end_id = min((k + 1) * batch, Total_data)
    num_of_data = end_id - stard_id
    # backward process
    with torch.inference_mode(), torch.amp.autocast(enabled=True, device_type=device_str, dtype=torch.bfloat16):
        #with torch.amp.autocast():
        x_hat = torch.randn(num_of_data, 1, 32, 32,device=device, dtype=dtype) *T
        T_time = torch.ones(len(x_hat), device=device, dtype=dtype) * T
    
        c_skip = lambda t: 0.25 / ((t - epsilon) ** 2 + 0.25)
        c_out = lambda t: np.sqrt(0.5 * (t - epsilon) / (t ** 2 + 0.25))
        x = c_skip(T) * x_hat + c_out(T) * f_theta(x_hat, T_time)
        # x = 0.5*x + 0.5
    
    # x_motion = torch.hstack([x_motion, x.unsqueeze(-1)])
    
    # result.append(x)   # Save the result
    resize_f = transforms.Resize((28,28))
    x_temp = resize_f(x).to(device="cpu", dtype=torch.float32).numpy().transpose((0, 2, 3, 1))  # [0][999]/
    x_temp = x.to(device="cpu", dtype=torch.float32).numpy().transpose((0, 2, 3, 1))  # [0][999]
    x_temp = np.clip(x_temp, 0.0, 1.0)
    # results.append((x_temp * 255).astype(np.uint8))


In [ ]:
N = 80

num_fig = 8
steps = int(N/5)
num_steps = int(N/steps)+1
# N=1000

fig = plt.figure(figsize=(num_fig, num_fig))

for j in range(num_fig**2):
    plt.subplot(num_fig, num_fig, j+1)     #将窗口分为两行两列四个子图，则可显示四幅图片
    # plt.title('Step = {}'.format(N - i))   #第一幅图片标题
    plt.imshow(x_temp[j], cmap = "gray")      #绘制第一幅图片
    plt.axis("off")
    k = k+1
    # plt.pause(1)

fig.tight_layout()
plt.subplots_adjust(wspace = 0, hspace = 0)

# plt.show()   #显示窗口
# plt.savefig("SM_sample_8")   #显示窗口

In [ ]:
# Plot reverse process

num_fig = 4
steps = 200
num_steps = int(N(0)/steps)+1

plt.figure(figsize=(2*num_steps, 2*num_fig))

for j in range(num_fig):
    k = 0
    for i in range(0, N(0)+1, steps):
        plt.subplot(num_fig, num_steps, k + num_steps*j+1)     #将窗口分为两行两列四个子图，则可显示四幅图片
        plt.title('Step = {}'.format(N(0) - i))   #第一幅图片标题
        plt.imshow(x_motion[j][i], cmap = "gray")      #绘制第一幅图片
        plt.axis("off")
        k = k+1
        # plt.pause(1)


# plt.show()   #显示窗口
# plt.savefig("SM_sample_8")   #显示窗口